# Análisis de Regresión (II)

En este cuaderno, veremos cómo el análisis de regresión puede ayudar a **entender el comportamiento de los datos**, **predecir valores de datos** (continuos o dicotómicos), y **encontrar predictores importantes** (modelos dispersos).
Presentamos diferentes modelos de regresión: regresión lineal simple, regresión lineal múltiple y regresión polinómica.
Evaluamos los resultados cualitativamente mediante herramientas de visualización de Seaborn y cuantitativamente mediante la biblioteca Scikit-learn, así como otras herramientas.

Usamos diferentes conjuntos de datos reales:
* Conjunto de datos macroeconómicos
* Predicción del precio de un nuevo mercado de viviendas
* Extensión del hielo marino y cambio climático
* Conjunto de datos de diabetes de Scikit-learn
* Conjunto de datos Longley de datos macroeconómicos de EE. UU.
* Conjunto de datos de publicidad

### Contenidos del cuaderno:

- Regresión Lineal Múltiple
- Regularización: Ridge y Lasso
- Transformación de Datos

In [ ]:
# Settings for the visualizations
import matplotlib.pylab as plt
%matplotlib inline 
plt.rc('font', size=12) 
plt.rc('figure', figsize = (12, 5))

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2,'font.family': [u'times']})

In [ ]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn.linear_model import LinearRegression


from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
seed = 1 # to make this notebook's output stable across runs

## Ejemplo 1: Vivienda en Boston

Continuemos con nuestro conjunto de datos de Vivienda en Boston.

In [ ]:
# Load dataset
from sklearn import datasets
boston = datasets.load_boston() # Dictionary-like object that exposes its keys as attributes.
X,y = boston.data, boston.target # Create X matrix and y vector from the dataset.
features = boston.feature_names
print('feature names: {}'.format(boston.feature_names))
print('Shape of data: {} {}'.format(X.shape, y.shape))

In [ ]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=seed)

print('Train and test sizes of X: {} {}'.format(X_train.shape, X_test.shape))
print('Train and test sizes of y: {} {}'.format(y_train.shape, y_test.shape))

In [ ]:
# Fitting a multiple linear model
lr = LinearRegression() # Create the Linear Regression estimator
lr.fit(X_train, y_train) # Perform the fitting


# Regrerssion coefs
coefs_lr = pd.Series(np.abs(lr.coef_), features).sort_values()

# Prediction
y_test_pred = lr.predict(X_test)

# evaluation
mse = mean_squared_error(y_test, y_test_pred)
r2score_train = lr.score(X_train, y_train)
r2score_test = lr.score(X_test, y_test)

# The coefficients
print('\nIntercept and coefs:\n{} {}'.format(lr.intercept_, lr.coef_))
# The mean squared error
print('\nMSE: {}'.format(mse))
# The coefficient of determination: 1 is perfect prediction
print('R^2 Score: {}'.format(r2score_train))
print('R^2 Score: {}'.format(r2score_test))

In [ ]:
# Plotting abs value of model coefficients
coefs_lr.plot(kind='bar', title='Model Coefficients')

In [ ]:
coefs_lr

Podemos ver que todos los coeficientes obtenidos son diferentes de cero, lo que significa que no se descarta ninguna variable.
A continuación, intentaremos construir un nuevo modelo para predecir el precio utilizando los factores más importantes y descartando los no informativos. Para hacer esto, podemos crear un regresor LASSO, forzando coeficientes a cero (ver más abajo).

In [ ]:
print(boston.DESCR)

## Modelos de Regularización

### Regularización L2: Regresión Ridge

La Regresión Ridge penaliza los coeficientes si estos están demasiado alejados de cero, obligándolos a ser pequeños de manera continua. De esta manera, reduce la complejidad del modelo mientras mantiene todas las variables en el modelo.

$$ minimize(\sum_{i=0}^n (y_i - \beta_0- \sum_{j=1}^p \beta_jx_{ij})^2 + \alpha\sum_{j=1}^p \beta_j^2) $$

donde $\beta_j$ son los coeficientes de regresión.


### Regularización L1: Regresión Lasso

A menudo, en problemas reales, hay variables no informativas en los datos que impiden un modelado adecuado del problema y, por lo tanto, la construcción de un modelo de regresión correcto. En tales casos, un proceso de selección de características es crucial para seleccionar solo las características informativas y descartar las no informativas. Esto se puede lograr mediante métodos dispersos que utilizan un enfoque de penalización, como *Lasso* (operador de encogimiento y selección absoluta mínima) para establecer algunos coeficientes del modelo a cero (descartando así esas variables). La dispersión puede verse como una aplicación de la navaja de Occam: preferir modelos más simples a los complejos.

Para ello, la regresión Lasso añade un término de regularización de **norma $\ell_1$** a la suma de errores cuadráticos de predicción (SSE). Dado el conjunto de muestras (𝑋,𝐲), el objetivo es minimizar:

$$ minimize(\sum_{i=0}^n (y_i - \beta_0- \sum_{j=1}^p \beta_jx_{ij})^2 + \alpha\sum_{j=1}^p|\beta_j|)$$

### Interpretación geométrica de la regularización

El panel izquierdo muestra la regularización L1 (regularización lasso) y el panel derecho la regularización L2 (regresión Ridge). Las elipses indican la distribución para no regularización. Las formas (cuadrado y círculo) muestran las restricciones debido a la regularización (limitando $\theta^2$ para la regresión Ridge y $|\theta|$ para la regresión Lasso). Las esquinas de la regularización L1 crean más oportunidades para que la solución tenga ceros en algunos de los pesos.

<center><img src="files/images/regularization-ridge-lasso.png"></center>

Más información [aquí](https://www.datacamp.com/community/tutorials/tutorial-ridge-lasso-elastic-net)

### Ridge


In [ ]:
## Ridge Regression
ridge = linear_model.Ridge(alpha=1) # Create a Ridge regressor
ridge.fit(X_train, y_train) # Perform the fitting

# Regrerssion coefs
coefs_ridge = pd.Series(np.abs(ridge.coef_), features).sort_values()

# Prediction
y_test_pred_ridge = ridge.predict(X_test)

# evaluation
mse_ridge = mean_squared_error(y_test, y_test_pred_ridge)
r2score_ridge_train = ridge.score(X_train, y_train)
r2score_ridge_test = ridge.score(X_test, y_test)

# The coefficients
print('\nIntercept and coefs:\n{} {}'.format(ridge.intercept_, ridge.coef_))
# The mean squared error
print('\nMSE: {}'.format(mse_ridge))
# The coefficient of determination: 1 is perfect prediction
print('R^2 Score train: {}'.format(r2score_ridge_train))
print('R^2 Score test: {}'.format(r2score_ridge_test))

In [ ]:
# Plotting abs value of model coefficients
coefs_ridge.plot(kind='bar', title='Ridge Coefficients')

### Lasso


**Para completar:**
Ajusta un regresor Lasso y evalúalo.

In [ ]:
## Lasso Regression
lasso = linear_model.Lasso(alpha=1)
lasso.fit(X_train, y_train)

# Regrerssion coefs
coefs_lasso = pd.Series(np.abs(lasso.coef_), features).sort_values()

# Prediction
y_test_pred_lasso = lasso.predict(X_test)

# evaluation
mse_lasso = mean_squared_error(y_test, y_test_pred_lasso)

r2score_lasso_train = lasso.score(X_train, y_train)
r2score_lasso_test = lasso.score(X_test, y_test)


# The coefficients

# The mean squared error

# The coefficient of determination: 1 is perfect prediction
print(mse_lasso)
print(r2score_lasso_train, r2score_lasso_test)



In [ ]:
# Plotting abs value of model coefficients
coefs_lasso.plot(kind='bar', title='Lasso Coefficients')


In [ ]:
coefs_lasso

#### Comparar los resultados:

In [ ]:
# Are the coeficients now sparse?
# Is the score different?

In [ ]:
f = plt.figure(figsize=(15,5))
ax1 = f.add_subplot(131)
ax2 = f.add_subplot(132)
ax3 = f.add_subplot(133)

coefs_lr.plot(kind="barh", title='coefs_lr', ax=ax1)
coefs_ridge.plot(kind="barh", title='coefs_ridge', ax=ax2)
coefs_lasso.plot(kind="barh", title='coefs_lasso', ax=ax3)

In [ ]:
print('Non important variables: {}'.format(coefs_lasso.index[coefs_lasso==0].values))
print('Most important variable: {}'.format(coefs_lasso.index[-1]))

In [ ]:
scores = [[r2score_train, r2score_test],
         [r2score_ridge_train, r2score_ridge_test],
         [r2score_lasso_train, r2score_lasso_test]]
df_scores = pd.DataFrame(scores, columns=["Train", "Test"], index=["No regularization", "Ridge", "Lasso"])
#df_scores.sort_values(by="test_score", ascending=False, inplace=True)
df_scores

## Transforma y Predice:

In [ ]:
# Look at mean and average values of our predictors
for i, feat in enumerate(features):
    print()
    print(feat)
    print("Max {}, min {}, mean {}, and var {}".format(np.max(X[:, i]), np.min(X[:, i]), np.mean(X[:, i]), np.var(X[:, i])))

Existe un tipo especial de ``Estimator`` llamado ``Transformer`` que transforma los datos de entrada, por ejemplo, selecciona un subconjunto de las características o extrae nuevas características basadas en las originales.

Un transformador que utilizaremos aquí es ``sklearn.preprocessing.StandardScaler``. Este transformador centra cada predictor en ``X`` para tener media cero y varianza unitaria y es útil.

In [ ]:
# Train test split
from sklearn.preprocessing import StandardScaler

scalerX = StandardScaler().fit(X_train) # Create the transformer StandardScaler and perform the fitting for the training data

X_train_norm = scalerX.transform(X_train)
X_test_norm = scalerX.transform(X_test)

print("\nBefore transformation:")
print('Train: Max {}, min {}, mean {}, and var {}'.format(np.max(X_train), np.min(X_train), np.mean(X_train), np.var(X_train)))
print('Test: Max {}, min {}, mean {}, and var {}'.format(np.max(X_test), np.min(X_test), np.mean(X_test), np.var(X_test)))

print("\nAfter transformation:")
print('Train: Max {}, min {}, mean {}, and var {}'.format(np.max(X_train_norm), np.min(X_train_norm), np.mean(X_train_norm), np.var(X_train_norm)))
print('Test: Max {}, min {}, mean {}, and var {}'.format(np.max(X_test_norm), np.min(X_test_norm), np.mean(X_test_norm), np.var(X_test_norm)))


In [ ]:
for i, feat in enumerate(features):
    print()
    print(feat)
    print("\nBefore transformation:")
    print('Train: Max {}, min {}, mean {}, and var {}'.format(np.max(X_train[:, i]), np.min(X_train[:, i]), np.mean(X_train[:, i]), np.var(X_train[:, i])))
    print('Test: Max {}, min {}, mean {}, and var {}'.format(np.max(X_test[:, i]), np.min(X_test[:, i]), np.mean(X_test[:, i]), np.var(X_test[:, i])))

    print("\nAfter transformation:")
    print('Train: Max {}, min {}, mean {}, and var {}'.format(np.max(X_train_norm[:, i]), np.min(X_train_norm[:, i]), np.mean(X_train_norm[:, i]), np.var(X_train_norm[:, i])))
    print('Test: Max {}, min {}, mean {}, and var {}'.format(np.max(X_test_norm[:, i]), np.min(X_test_norm[:, i]), np.mean(X_test_norm[:, i]), np.var(X_test_norm[:, i])))


Ahora comparemos los coeficientes que obtendríamos si usáramos la matriz de variables estandarizadas en su lugar.

In [ ]:
# Train model
lr_norm = linear_model.LinearRegression()
ridge_norm = linear_model.Ridge(alpha=.3)
lasso_norm = linear_model.Lasso(alpha=.3)

lr_norm.fit(X_train_norm, y_train)
ridge_norm.fit(X_train_norm, y_train)
lasso_norm.fit(X_train_norm, y_train)

coefs_lr_norm = pd.Series(np.abs(lr_norm.coef_), boston.feature_names).sort_values()
coefs_ridge_norm = pd.Series(np.abs(ridge_norm.coef_), boston.feature_names).sort_values()
coefs_lasso_norm = pd.Series(np.abs(lasso_norm.coef_), boston.feature_names).sort_values()

In [ ]:
f = plt.figure(figsize=(15,5))
ax1 = f.add_subplot(131)
ax2 = f.add_subplot(132)
ax3 = f.add_subplot(133)

coefs_lr.plot(kind="barh", title='coefs_lr', ax=ax1)
coefs_ridge.plot(kind="barh", title='coefs_ridge', ax=ax2)
coefs_lasso.plot(kind="barh", title='coefs_lasso', ax=ax3)

In [ ]:
f = plt.figure(figsize=(15,5))
ax1 = f.add_subplot(131)
ax2 = f.add_subplot(132)
ax3 = f.add_subplot(133)

coefs_lr_norm.plot(kind="barh", title='coefs_lr_norm', ax=ax1)
coefs_ridge_norm.plot(kind="barh", title='coefs_ridge_norm', ax=ax2)
coefs_lasso_norm.plot(kind="barh", title='coefs_lasso_norm', ax=ax3)

In [ ]:
print('Non important variables:')
print('Before transformation: {}'.format(sorted(coefs_lasso.index[coefs_lasso_norm==0].values)))
print('After transformation: {}'.format(sorted(coefs_lasso_norm.index[coefs_lasso_norm==0].values)))
print('Most important variable:')
print('Before transformation: {}'.format(coefs_lasso.index[-1]))
print('After transformation: {}'.format(coefs_lasso_norm.index[-1]))

In [ ]:
# Scores
print('lr: {}'.format(lr.score(X_test, y_test)))
print('ridge: {}'.format(ridge.score(X_test, y_test)))
print('lasso: {}'.format(lasso.score(X_test, y_test)))
print('lr_norm: {}'.format(lr_norm.score(X_test_norm, y_test)))
print('ridge_norm: {}'.format(ridge_norm.score(X_test_norm, y_test)))
print('lasso_norm: {}'.format(lasso_norm.score(X_test_norm, y_test)))

### Ajustando los regresores Ridge y Lasso

In [ ]:
n_alphas = 100
alphas = np.logspace(-2, 2, n_alphas)

coefs_ridge = []
r2_ridge = []
for l in alphas:
    regr_ridge = linear_model.Ridge(alpha=l) # Create a Ridge regressor
    regr_ridge.fit(X_train_norm, y_train)  # Perform the fitting
    coefs_ridge.append(regr_ridge.coef_)
    r2_ridge.append(regr_ridge.score(X_test_norm,y_test))

    
coefs_lasso = []
r2_lasso = []
for l in alphas:
    regr_lasso = linear_model.Lasso(alpha=l,tol =0.001) # Create a Ridge regressor
    regr_lasso.fit(X_train_norm, y_train)  # Perform the fitting
    coefs_lasso.append(regr_lasso.coef_)
    r2_lasso.append(regr_lasso.score(X_test_norm,y_test))

In [ ]:
# Display results

fig, axs = plt.subplots(2, 2, figsize=(20, 20), sharey='row')


axs[0,0].plot(alphas, np.abs(coefs_ridge))
axs[0,0].set_xscale('log')
axs[0,0].set_title('Ridge coefficients as a function of the regularization')
axs[0,0].axis('tight')
axs[0,0].set_xlabel('alpha')
axs[0,0].set_ylabel('weights')
axs[0,0].legend(boston.feature_names)

axs[0,1].plot(alphas, np.abs(coefs_lasso))
axs[0,1].set_xscale('log')
axs[0,1].set_title('Lasso coefficients as a function of the regularization')
axs[0,1].axis('tight')
axs[0,1].set_xlabel('alpha')
axs[0,1].set_ylabel('weights')
axs[0,1].legend(boston.feature_names)

axs[1,0].plot(alphas, r2_ridge)
axs[1,0].set_xscale('log')
axs[1,0].set_title('Ridge scores as a function of the regularization')
axs[1,0].axis('tight')
axs[1,0].set_xlabel('alpha')
axs[1,0].set_ylabel('r2')

axs[1,1].plot(alphas, r2_lasso)
axs[1,1].set_xscale('log')
axs[1,1].set_title('Lasso scores as a function of the regularization')
axs[1,1].axis('tight')
axs[1,1].set_xlabel('alpha')
axs[1,1].set_ylabel('r2')


plt.show()

In [ ]:
# Find optimal alphas
best_r2_ridge = max(r2_ridge)
max_index_ridge = r2_ridge.index(best_r2_ridge)
best_alpha_ridge = alphas[max_index_ridge]
print(max_index_ridge, best_alpha_ridge, best_r2_ridge, r2_ridge[max_index_ridge])

best_r2_lasso = max(r2_lasso)
max_index_lasso = r2_lasso.index(best_r2_lasso)
best_alpha_lasso = alphas[max_index_lasso]
print(max_index_lasso, best_alpha_lasso, best_r2_lasso, r2_lasso[max_index_lasso])

In [ ]:
print(r2score_test, r2score_train)

In [ ]:
lasso = linear_model.Lasso(alpha=best_alpha_lasso)
lasso.fit(X_train_norm, y_train)
coefs = pd.Series(np.abs(lasso.coef_), features).sort_values()

In [ ]:
coefs

In [ ]:
df = pd.DataFrame(X_train_norm, columns=features)

In [ ]:
df['targ'] = y_train

In [ ]:
features

In [ ]:
sns.pairplot(df[['RM', 'NOX', 'DIS', 'PTRATIO', 'targ']])

### Selección de características con Sklearn

También podemos seleccionar las características más importantes con sklearn:

In [ ]:
import sklearn.feature_selection as fs 
selector = fs.SelectKBest(score_func = fs.f_regression, k=5)

X_new_train = selector.fit_transform(X_train,y_train)
X_new_test = selector.transform(X_test)

print('Non important variables: {}'.format(boston.feature_names[selector.get_support()==False]))
print('Relevant variables: {}'.format(boston.feature_names[selector.get_support()]))

In [ ]:
X_new_train.shape

El conjunto de características seleccionadas ahora es diferente, ya que el criterio ha cambiado.

El método SelectKBest selecciona características según las k puntuaciones más altas. La puntuación se calcula usando la función score_func.
En este caso, utilizamos f_regression como nuestra función de puntuación, que devuelve la estadística F y los valores p de las pruebas de regresión lineal univariante de cada característica en X contra y.

**EJERCICIO 3** Diabetes:

El conjunto de datos de diabetes (de scikit-learn) consta de 10 variables fisiológicas (edad, sexo, peso, presión arterial) medidas en 442 pacientes, y una indicación de la progresión de la enfermedad después de un año.

Exploraremos el rendimiento del modelo de Regresión Lineal y el modelo LASSO para la predicción.

Completa los huecos del ejercicio.

Carga los datos

In [ ]:
from sklearn import datasets
diabetes = datasets.load_diabetes()
X,y = diabetes.data, diabetes.target
print(X.shape, y.shape)

In [ ]:
features = diabetes.feature_names
features

Evalúa la predicción usando un modelo de regresión simple y un modelo de regresión múltiple.

In [ ]:
df = pd.DataFrame(X, columns=features)
df['targ'] = y
df.head()

In [ ]:
corr_mat = np.abs(df.corr())
sns.heatmap(corr_mat)

In [ ]:
sns.pairplot(data=df[['bmi', 's5', 'bp', 's4', 'targ']])

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2)
print(df_train.shape, df_test.shape)

Para el modelo simple, primero elige una de las dimensiones de los datos. Intenta realizar algunos gráficos para identificar posibles relaciones lineales entre las variables predictoras y las variables objetivo. Elige una variable para tu primer modelo.

In [ ]:
X_train = df_train[['s5']]
y_train = df_train['targ']
print(X_train.shape, y_train.shape)

In [ ]:
X_test = df_test[['s5']]
y_test = df_test['targ']
print(X_test.shape, y_test.shape)

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)
print(lm.score(X_train, y_train), lm.score(X_test, y_test))

Divide en conjuntos de entrenamiento y prueba y evalúa la predicción (sklearn) con un modelo de regresión múltiple.

In [ ]:
X_train = df_train.drop(columns='targ')
y_train = df_train['targ']

X_test = df_test.drop(columns='targ')
y_test = df_test['targ']


print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)
print(lm.score(X_train, y_train), lm.score(X_test, y_test))

print(lm.coef_)

Para el modelo de regresión múltiple, divide en conjuntos de entrenamiento y prueba y evalúa la predicción (sklearn) sin y con regularización LASSO.

In [ ]:
scaler = StandardScaler()
X_train_new = scaler.fit_transform(X_train)
X_test_new = scaler.transform(X_test)

In [ ]:
lm = linear_model.Lasso(alpha=1)
lm.fit(X_train_new, y_train)
print(lm.score(X_train_new, y_train), lm.score(X_test_new, y_test))

In [ ]:
pd.Series(np.abs(lm.coef_), features).sort_values()

 Almost the same results with less "activated" coefficients (the result has 3 zero coefficients).
 
 Is the score different? How many predictors are we using now?


**EJERCICIO 4: Ventas de Big Mart**

Usa el [conjunto de datos de ventas de Big Mart](https://www.kaggle.com/brijbhushannanda1979/bigmart-sales-data). En el conjunto de datos, tenemos ventas de productos por producto para múltiples sucursales de una cadena.

En particular, podemos ver características del artículo vendido (contenido de grasa, visibilidad, tipo, precio) y algunas características de la sucursal (año de establecimiento, tamaño, ubicación, tipo) y el número de artículos vendidos para ese artículo en particular. Veamos si podemos predecir las ventas usando estas características.

Implementa el siguiente análisis:
- Lee los archivos de entrenamiento y prueba en un DataFrame de pandas
- Limpia los datos (hay algunos valores faltantes)
- Convierte las variables categóricas en valores numéricos y excluye 'Item_Identifier' y 'Item_Outlet_Sales' (que es el objetivo).
- Estudia cuáles son las variables con mayor (menor) correlación con la variable objetivo.
- Aplica regresión lineal usando todas las características.
- Construye el gráfico de residuos y da una interpretación del mismo
- Elige un modelo de regresión polinómica para ajustar mejor los datos.
- Compara los regresores de ridge y lasso.
- Compara la magnitud de los coeficientes de los diferentes modelos.
- Estima cuáles son las mejores características para la predicción.

Lee los archivos de entrenamiento y prueba en un DataFrame de pandas

In [ ]:
# Load data:
df_train = pd.read_csv('files/ch06/bigmart-sales-data/Train.csv')

df_test = pd.read_csv('files/ch06/bigmart-sales-data/test.csv')

df_train.head()

Limpia los datos (hay algunos valores faltantes).

In [ ]:
df_train.info()

Observa los datos faltantes en Item_Weight y Outlet_Size. Veamos cómo se ven estas variables.

In [ ]:
print(df_train.Item_Weight.mean(), df_train.Item_Weight.std())
plt.hist(df_train.Item_Weight, bins=20)

In [ ]:
# Replace nulls Item_Weight in with mean
mean_Item_Weight = df_train.Item_Weight.mean()
df_train2 =  df_train.copy()
df_test2 =  df_test.copy()

print(mean_Item_Weight)
df_train2[['Item_Weight']] = df_train2[['Item_Weight']].fillna(value=mean_Item_Weight)
df_test2[['Item_Weight']] = df_test2[['Item_Weight']].fillna(value=mean_Item_Weight)
df_train2[df_train.Item_Weight.isna()]

In [ ]:
sns.countplot(x="Outlet_Size", data=df_train2)

In [ ]:
df_train2[['Outlet_Size']].value_counts()

In [ ]:
# We'll fill the empty values of Outlet_Size with medium, since it´s the most common value.
df_train2[['Outlet_Size']] = df_train2[['Outlet_Size']].fillna(value='Medium')
df_test2[['Outlet_Size']] = df_test2[['Outlet_Size']].fillna(value='Medium')
df_train2[['Outlet_Size']].value_counts()

Convierte las variables categóricas en valores numéricos y excluye 'Item_Identifier' y 'Item_Outlet_Sales' (que es el objetivo).

In [ ]:
y = df_train2['Item_Outlet_Sales']

df_train2.drop(columns=['Item_Identifier','Item_Outlet_Sales'], inplace=True)
df_test2.drop(columns=['Item_Identifier'], inplace=True)

In [ ]:
df_train2.info()

In [ ]:
cols_num = ['Item_Weight', 'Item_Visibility', 'Item_MRP','Outlet_Establishment_Year']
cols_cat = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

In [ ]:
for col in cols_cat:
    print()
    print(df_train2[[col]].value_counts())

In [ ]:
df_train2 = pd.get_dummies(df_train2, drop_first=True)
df_test2 = pd.get_dummies(df_test2, drop_first=True)
df_test2

Estudia cuáles son las variables con la mayor (menor) correlación con la variable objetivo.

In [ ]:
cols_num = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year']
X_num = df_train2[cols_num].values
X_num_test = df_test2[cols_num].values
(X_num.shape, X_num_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

scalerX = StandardScaler().fit(X_num) # Create the transformer StandardScaler and perform the fitting for the training data

X_num = scalerX.transform(X_num)
X_num_test = scalerX.transform(X_num_test)

In [ ]:
df_train2

In [ ]:
df_train2[cols_num] = X_num
df_test2[cols_num] = X_num_test
df_train2

In [ ]:
df_train2['y'] = y
corr_mat = np.abs(df_train2.corr())

fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(corr_mat, square=True, ax=ax)

In [ ]:
corr_mat.y.sort_values(ascending=False)

Aplica regresión lineal utilizando todas las características.

In [ ]:
X = df_train2.drop(columns=['y'])
X.shape

In [ ]:
lr = LinearRegression()
lr.fit(X, y)

y_pred = lr.predict(X)

print(lr.score(X, y))

Construye el gráfico de residuos y proporciona una interpretación del mismo.

In [ ]:
sns.scatterplot(x = y_pred, y = y - y_pred, alpha=0.4)

plt.hlines(y=0, xmin= 0, xmax=y_pred.max())
plt.title('Residual plot')
plt.xlabel('$\hat y$')
plt.ylabel('$y - \hat y$')

In [ ]:
plt.scatter(y, y_pred, alpha=0.3)

plt.plot([y.min(), y.max()], [y.min(), y.max()], '--k')
plt.axis('tight')
plt.xlabel('$y$')
plt.ylabel('$\hat y$')

Elige un modelo de regresión polinómica para ajustar mejor los datos.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
poly = PolynomialFeatures(degree=2)
X2 = poly.fit_transform(X)

clf = linear_model.LinearRegression()
clf.fit(X2, y)

print(clf.score(X2, y))

Compara los regresores de ridge y lasso.

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import Ridge, Lasso
#lr = LinearRegression()
#lr.fit(X, y)

ridge = Ridge(alpha=.5)
ridge.fit(X, y)

lasso = Lasso(alpha=.5)
lasso.fit(X, y)

print(lr.score(X, y), ridge.score(X, y), lasso.score(X, y))

Compara la magnitud de los coeficientes de los diferentes modelos.

In [ ]:
coefs_lr = pd.Series(lr.coef_, df_train2.columns[:-1]).sort_values()
coefs_ridge = pd.Series(ridge.coef_, df_train2.columns[:-1]).sort_values()
coefs_lasso = pd.Series(lasso.coef_, df_train2.columns[:-1]).sort_values()

In [ ]:
print(coefs_lr)

In [ ]:
print(coefs_ridge)

In [ ]:
print(coefs_lasso)

Estima cuáles son las mejores características para la predicción.

In [ ]:
features = df_train2.columns[:-1]
features

In [ ]:
import sklearn.feature_selection as fs 
selector = fs.SelectKBest(score_func = fs.f_regression,k=5)

X_new = selector.fit_transform(X,y)

print('Non important variables: {}'.format(features[selector.get_support()==False]))
print('Relevant variables: {}'.format(features[selector.get_support()]))

**ANÁLISIS ADICIONAL PARA LOS DATOS DE BOSTON**

Para comparar el ajuste de los modelos de regresión lineal y polinómica también podemos usar la biblioteca sklearn.

A continuación, añadimos una evaluación cuantitativa de los dos modelos.

In [ ]:
boston = datasets.load_boston()
X_boston,y_boston = boston.data, boston.target

In [ ]:
# Evaluation of the linear model
X_boston,y_boston = boston.data, boston.target

regr_boston = LinearRegression()
regr_boston.fit(X_boston, y_boston) 

#print('Coeff and intercept: {} {}'.format(regr_boston.coef_, regr_boston.intercept_))
print('Multiple Linear regression Score: {}'.format(regr_boston.score(X_boston, y_boston)))
print('Multiple Linear regression MSE: {}'.format(np.mean((regr_boston.predict(X_boston) - y_boston)**2)))

In [ ]:
# Evaluation of the polynomial model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

regr_pol = Pipeline([('poly', PolynomialFeatures(degree=2)),('linear', LinearRegression(fit_intercept=False))])
regr_pol.fit(X_boston, y_boston) 

#print('Coeff and intercept: {} {}'.format(regr_pol.named_steps['linear'].coef_, regr_pol.named_steps['linear'].intercept_))
print('Multiple Polynomial regression Score: {}'.format(regr_pol.score(X_boston, y_boston)))
print('Multiple Polynomial regression MSE: {}'.format(np.mean((regr_pol.predict(X_boston) - y_boston)**2)))

Para la regresión simple, primero necesitamos extraer una de las características y luego usar los mismos métodos:

In [ ]:
# Quantitative evaluation of the SIMPLE lineal and polynomial regression:
bostonDF = pd.DataFrame(boston.data)
bostonDF.head()

In [ ]:
bostonDF.columns=boston.feature_names 
bostonDF.head()

In [ ]:
x=bostonDF['LSTAT']
y=boston.target
x = np.expand_dims(x, axis=1)
y = np.expand_dims(y, axis=1)

In [ ]:
regr_boston = LinearRegression()
regr_boston.fit(x, y) 

print('Simple linear regression Score: {}'.format(regr_boston.score(x, y)))
print('Simple linear regression MSE: {}'.format(np.mean((regr_boston.predict(x) - y)**2)))

regr_pol = Pipeline([('poly', PolynomialFeatures(degree=2)),('linear', LinearRegression(fit_intercept=False))])
regr_pol.fit(x, y) 

print('Simple Polynomial regression (order 2) Score: {}'.format(regr_pol.score(x, y)))
print('Simple Polynomial regression (order 2) MSE: {}'.format(np.mean((regr_pol.predict(x) - y)**2)))

regr_pol = Pipeline([('poly', PolynomialFeatures(degree=3)),('linear', LinearRegression(fit_intercept=False))])
regr_pol.fit(x, y) 

print('Simple Polynomial regression (order 3) Score: {}'.format(regr_pol.score(x, y)))
print('Simple Polynomial regression (order 3) MSE: {}'.format(np.mean((regr_pol.predict(x) - y)**2)))

**EJERCICIO 2: Conjunto de datos macroeconómicos**

Para comenzar, cargamos el conjunto de datos Longley de datos macroeconómicos de EE. UU. desde el sitio web de conjuntos de datos de R. Datos macroeconómicos desde 1947 hasta 1962.

In [ ]:
# Read data
df = pd.read_csv('http://vincentarelbundock.github.io/Rdatasets/csv/datasets/longley.csv', index_col=0)
df.head()

# Clean column names
df.columns = ['GNPdeflator', 'GNP', 'Unemployed', 'ArmedForces', 'Population','Year', 'Employed']
features = ['GNPdeflator','Unemployed','ArmedForces','Population','Year','Employed']
target = 'GNP'

# Create X matrix and y vector from the dataset
X = df[features].values
y = df[target].values

print('Shape of data: {} {}'.format(X.shape, y.shape))

In [ ]:
# Fitting a multiple linear model
lin_reg = LinearRegression() # Create the Linear Regression estimator
lin_reg.fit(X, y) # Perform the fitting


# Regrerssion coefs
coefs = pd.Series(lin_reg.coef_, features).sort_values()

# Prediction
y_pred = lin_reg.predict(X)

# evaluation
mse = mean_squared_error(y, y_pred)
r2score = lin_reg.score(X, y)

# The coefficients
print('\nIntercept and coefs:\n{} {}'.format(lin_reg.intercept_, lin_reg.coef_))
# The mean squared error
print('\nMSE: {}'.format(mse))
# The coefficient of determination: 1 is perfect prediction
print('R^2 Score: {}'.format(r2score))

In [ ]:
# Plotting abs value of model coefficients
np.abs(coefs).sort_values().plot(kind='bar', title='Model Coefficients')